In [1]:
# third party

# syft absolute
import syft as sy

sy.__version__

'0.8.6-beta.1'

In [2]:
try:
    gw_node.land()
    dm_node1.land()
    dm_node2.land()
except Exception:
    pass

In [3]:
web_server_node = False

if web_server_node:
    # Launch a fresh domain server named "test-domain-1" in dev mode on the local machine
    gw_node = sy.orchestra.launch(
        name="test-gateway-1", port=9000, dev_mode=True, reset=True, node_type="gateway"
    )
    dm_node1 = sy.orchestra.launch(
        name="test-domain-1", port=9001, dev_mode=True, reset=True, node_type="domain"
    )
    dm_node2 = sy.orchestra.launch(
        name="test-domain-2", port=9002, dev_mode=True, reset=True, node_type="domain"
    )
else:
    gw_node = sy.orchestra.launch(
        name="test-gateway-1", dev_mode=True, reset=True, node_type="gateway"
    )
    dm_node1 = sy.orchestra.launch(
        name="test-domain-1", dev_mode=True, reset=True, node_type="domain"
    )
    dm_node2 = sy.orchestra.launch(
        name="test-domain-2", dev_mode=True, reset=True, node_type="domain"
    )

gw_client = gw_node.login(email="info@openmined.org", password="changethis")
dm_client1 = dm_node1.login(email="info@openmined.org", password="changethis")
dm_client2 = dm_node2.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=d2641d00ec504de7828b332929c6cfa8 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=93f16fb7532a4f4e9f90dfeaef453324 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=9ea1cec8b0fc4eaa93da030e589d74b0 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <test-gateway-1: High side Gateway> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test-domain-2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
dm_client1.connect_to_gateway(gw_client)
dm_client2.connect_to_gateway(gw_client)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=93f16fb7532a4f4e9f90dfeaef453324 in_memory=True
Default worker pool error. Invalid number of workers: 0
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=d2641d00ec504de7828b332929c6cfa8 in_memory=True
Default worker pool error. Invalid number of workers: 0
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=93f16fb7532a4f4e9f90dfeaef453324 in_memory=True
Default worker pool error. Invalid number of workers: 0
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=d2641d00ec504de7828b332929c6cfa8 in_memory=True


SyftSuccess: Connected domain to test-gateway-1 gateway

In [5]:
assert len(gw_client.peers) == 2
gw_client.peers

syft.client.gateway_client.ProxyClient

In [6]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute]

In [7]:
gw_peers[1].node_routes

[syft.service.network.routes.PythonNodeRoute]

#### Adding an existed route should update its priority

In [8]:
assert gw_peers[0].node_routes[0].priority == 1
gw_peers[0].node_routes[0].priority

1

In [9]:
# stdlib
from copy import deepcopy

# syft absolute
from syft.types.uid import UID

existed_route = deepcopy(gw_peers[0].node_routes[0])
existed_route.id = UID()

In [10]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=existed_route)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=93f16fb7532a4f4e9f90dfeaef453324 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: The route already exists with id 2c50dba106134f5f84995c684fb91338, so its priority was updated

In [11]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute]

In [12]:
gw_peers = gw_client.api.services.network.get_all_peers()
assert gw_peers[0].node_routes[0].priority == 2
gw_peers[0].node_routes[0].priority

2

#### Adding a new route

In [13]:
if web_server_node:
    # syft absolute
    from syft.service.network.routes import HTTPNodeRoute

    new_route = HTTPNodeRoute(host_or_ip="localhost", port=10000)

else:
    # syft absolute
    from syft.node.worker_settings import WorkerSettings
    from syft.service.network.routes import PythonNodeRoute

    worker_settings = WorkerSettings.from_node(node=gw_node.python_node)
    worker_settings.node_side_type = "low"
    new_route = PythonNodeRoute(worker_settings=worker_settings)

new_route

```python
class PythonNodeRoute:
  id: str = fe678e16779144c5a6e1418052a0ccd7

```

In [14]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=new_route)

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=0 image_uid=93f16fb7532a4f4e9f90dfeaef453324 in_memory=True
Default worker pool error. Invalid number of workers: 0


SyftSuccess: New route with id 'fe678e16779144c5a6e1418052a0ccd7' added for test-domain-1

In [15]:
gw_peers = gw_client.api.services.network.get_all_peers()
assert len(gw_peers[0].node_routes) == 2
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

In [16]:
for route in gw_peers[0].node_routes:
    print(f"route {route.id} has priority = {route.priority}")

route 2c50dba106134f5f84995c684fb91338 has priority = 2
route fe678e16779144c5a6e1418052a0ccd7 has priority = 3


#### Delete a route
Delete using route's id

In [17]:
gw_client.api.services.network.delete_route_for(peer=gw_peers[0], route_id=new_route.id)

SyftSuccess: Route fe678e16779144c5a6e1418052a0ccd7 deleted for peer test-domain-1!

In [18]:
gw_peers = gw_client.api.services.network.get_all_peers()
assert len(gw_peers[0].node_routes) == 1
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute]

Delete using the `NodeRoute` object

In [19]:
gw_peers[0].node_routes[0]

```python
class PythonNodeRoute:
  id: str = 2c50dba106134f5f84995c684fb91338

```

In [20]:
gw_client.api.services.network.delete_route_for(
    peer=gw_peers[0], route=gw_peers[0].node_routes[0]
)

SyftSuccess: Route 2c50dba106134f5f84995c684fb91338 deleted for peer test-domain-1! No routes left for peer test-domain-1, so it is deleted!

In [21]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers

[syft.service.network.node_peer.NodePeer]

#### Update a route's priority

In [ ]:
if web_server_node:
    # syft absolute
    from syft.service.network.routes import HTTPNodeRoute

    new_route = HTTPNodeRoute(host_or_ip="localhost", port=10020)

else:
    # syft absolute
    from syft.node.worker_settings import WorkerSettings
    from syft.service.network.routes import PythonNodeRoute

    worker_settings = WorkerSettings.from_node(node=gw_node.python_node)
    worker_settings.node_side_type = "low"
    new_route = PythonNodeRoute(worker_settings=worker_settings)

new_route

In [ ]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

In [ ]:
gw_client.api.services.network.add_route_for(peer=gw_peers[0], route=new_route)

In [29]:
gw_peers = gw_client.api.services.network.get_all_peers()
gw_peers[0].node_routes

[syft.service.network.routes.PythonNodeRoute,
 syft.service.network.routes.PythonNodeRoute]

Not specifying new priority

In [30]:
gw_client.api.services.network.update_route_priority_for(
    peer=gw_peers[0], route=new_route
)

SyftError: Not implemented

When specifying new priority

In [24]:
gw_client.api.services.network.update_route_priority_for(
    peer=gw_peers[0], route=new_route, priority=3
)

SyftError: Not implemented

### Launch the first domain node 1 again but with different port

In [ ]:
dm_node1 = sy.orchestra.launch(
    name="test-domain-1", port=9098, dev_mode=True, reset=True, node_type="domain"
)
dm_client1 = dm_node1.login(
    email="info@openmined.org", password="changethis", port=9098
)

In [ ]:
dm_client1.connect_to_gateway(gw_client)

In [ ]:
dm_client1.peers[0].node_routes

In [ ]:
gw_client.peers

In [ ]:
dm_client1.peers